In [ ]:
# Importamos las librerias necesarias
import pandas as pd
from pandasql import sqldf

In [ ]:
# Importamos los archivos CSV, uno para cada servicio de streaming
Amazon = pd.read_csv("https://raw.githubusercontent.com/adelgerbo/Datasets/main/amazon_prime_titles-score.csv")
Disney = pd.read_csv("https://raw.githubusercontent.com/adelgerbo/Datasets/main/disney_plus_titles-score.csv")
Hulu = pd.read_csv("https://raw.githubusercontent.com/adelgerbo/Datasets/main/hulu_titles-score%20(2).csv")
Netflix = pd.read_csv("https://raw.githubusercontent.com/adelgerbo/Datasets/main/netflix_titles-score.csv")

In [ ]:
# Verificamos la importaciòn correcta de los datasets
Amazon.head()

In [ ]:
Disney.head()

In [ ]:
Hulu.head()

In [ ]:
Netflix.tail()

In [ ]:
"""
Generamos el campo id: Cada id se compondrá de la primera letra del
nombre de la plataforma, seguido del show_id ya presente en los 
datasets (ejemplo para títulos de Amazon = as123)
"""
Amazon.insert(loc = 0,
          column = 'id',
          value = "a"+Amazon["show_id"])

Disney.insert(loc = 0,
          column = 'id',
          value = "d"+Disney["show_id"])

Hulu.insert(loc = 0,
          column = 'id',
          value = "h"+Hulu["show_id"])

Netflix.insert(loc = 0,
          column = 'id',
          value = "n"+Netflix["show_id"])          



In [ ]:
# Comprobamos la correcta inserciòn del nuevo campo
Netflix.head()


In [ ]:
"""
Juntamos todos los datasets en uno solo, ya que contamos 
con el campo id para diferenciar el servicio, y las
transformaciones a continuación se aplican a todos por igual
"""

moviesdb = Amazon.append(Disney).append(Hulu).append(Netflix)

In [ ]:
#Verificamos el resultado
moviesdb.head()

In [ ]:
moviesdb.tail()

In [ ]:
# Verificamos existencia y cantidad de nulos por columna en el dataframe
moviesdb.isnull().sum()

In [ ]:
"""
Reemplazamos los valores nulos del campo "Rating" por la letra "G"
(corresponde al maturity rating "General por all audiences")
"""
moviesdb["rating"].fillna("G", inplace = True)

In [ ]:
# Controlamos la correcciòn sobre la columna "rating"
moviesdb.isnull().sum()

In [ ]:
# Visualizamos el dataframe modificado
moviesdb.head(30)

In [ ]:
# Cambiamos el formato de la columna "date_added" a AAAA-mm-dd
moviesdb["date_added"] = pd.to_datetime(moviesdb["date_added"])


In [ ]:
# Verificamos
moviesdb.tail()

In [ ]:
# Pasamos a minúsculas los campos de texto
moviesdb["type"] = moviesdb["type"].str.lower()
moviesdb["title"] = moviesdb["title"].str.lower()
moviesdb["director"] = moviesdb["director"].str.lower()
moviesdb["cast"] = moviesdb["cast"].str.lower()
moviesdb["country"] = moviesdb["country"].str.lower()
moviesdb["rating"] = moviesdb["rating"].str.lower()
moviesdb["duration"] = moviesdb["duration"].str.lower()
moviesdb["listed_in"] = moviesdb["listed_in"].str.lower()
moviesdb["description"] = moviesdb["description"].str.lower()


In [ ]:
# Verificamos
moviesdb.head()

In [ ]:
# Separamos el campo "duration" en 2 columnas
splitcolumns = moviesdb["duration"].str.split(" ", n = 1, expand = True)
 
# Insertamos una columna con las cantidades
moviesdb["duration_int"] = splitcolumns[0]
 
# Insertamos una columna con la unidad de medición
moviesdb["duration_type"] = splitcolumns[1]


In [ ]:
# Reemplazamos "seasons" por "season" en la columna "duration_type"
moviesdb['duration_type'] = moviesdb['duration_type'].str.replace('seasons','season')

In [ ]:
# Cambiamor el formato del campo "duration_int" a integer
moviesdb['duration_int'] = moviesdb['duration_int'].astype("Int64")

In [ ]:
#Verificamos
moviesdb.head(10)

In [ ]:
moviesdb.info()

In [ ]:
"""
Comenzamos con las consultas al dataframe. Esta función nos evitará pasar variables
globales cada vez que usemos un objeto
"""
pysqldf = lambda q: sqldf(q, globals())


In [ ]:
"""
Definimos la función para el siguiente requerimiento: Cantidad de títulos 
conteniendo una keyword, por plataforma
"""
@app.get("/get_word_count/{plataforma}/{keyword}")
def get_word_count(keyword:str, plataforma:str):
    if plataforma == "amazon":
        plat = "a%"
    elif plataforma == "netflix":
        plat = "n%"
    elif plataforma == "hulu":
        plat = "h%"
    elif plataforma == "disney":
        plat = "d%"
    else:
        return "No ha seleccionado la plataforma entre las opciones posibles"

    query = (
        """SELECT COUNT(title)
        FROM moviesdb
        WHERE title LIKE '"""+keyword+"""'
        AND id LIKE '"""
        + plat
        + """' """
    )
    veces = pysqldf(query)
    return veces.to_string(index=False, header=False)

In [ ]:
"""
Definimos la función para el siguiente requerimiento: Cantidad de películas 
por plataforma con un puntaje mayor a XX en determinado año
"""
@app.get("/get_score_count/{plataforma}/{score}/{anio}")
def get_score_count(plataforma:str, score:int, anio:int):
    if plataforma == "amazon":
        plat = "a%"
    elif plataforma == "netflix":
        plat = "n%"
    elif plataforma == "hulu":
        plat = "h%"
    elif plataforma == "disney":
        plat = "d%"
    else:
        return "No ha seleccionado la plataforma entre las opciones posibles"

query = """SELECT COUNT(title)
        FROM moviesdb
        WHERE score > """+score+"""
        AND release_year = """+anio+"""
        AND id LIKE '"""
        + plat
        + """'
        AND type = "movie" """
cantidad = pysqldf(query)
return cantidad.to_string(index=False, header=False)

In [ ]:
query = """SELECT title
        FROM moviesdb
        WHERE id LIKE "a%"
        AND type = "movie"
        ORDER BY score DESC, title ASC
        LIMIT 1,1"""
veces = pysqldf(query)
veces

In [ ]:
query = """SELECT title
        FROM moviesdb
        WHERE id LIKE "n%"
        AND release_year = 2016
        AND duration_type = "min"
        AND duration_int = (SELECT MAX(duration_int)
        FROM moviesdb
        WHERE id LIKE "n%"
        AND release_year = 2016
        AND duration_type = "min")
        """
veces = pysqldf(query)
veces

In [ ]:
query = """SELECT count(title)
        FROM moviesdb
        WHERE rating = "all" """
veces = pysqldf(query)
veces